####**Sebastián Numpaque - C. C. 1002396960**

#**Minitarea 6 - SPH y el perfil de Densidad y Presión de una Estrella**

Usando el código de SPH visto en clase, dibujar el perfil de densidad y presión de una nube de gas autogravitante.

Para ello:
- Escoja una masa para la nube y use los mismos parámetros usados en clase.

- Integre las posiciones y velocidades hasta que las velocidades de las partículas sean menores a una tolerancia específica (defina usted la tolerancia). En ese momento asumiremos que alcanza el equilibrio hidrostático.

- Calcule el perfil de densidad y presión para la nube tomando al menos 10 direcciones diferentes desde el origen hasta la superficie y promediando los perfiles obtenidos.

- Cambie el valor de la masa y determine: el tiempo que tarda en alcanzar el equilibrio, los perfiles de densidad y presión.



---



#**Cloning GitHub Repository**

Smoothed-Particle Hydrodynamics Simulation of Toy Star

- https://github.com/pmocz/sph-python/blob/master/sph.py

In [1]:
# @title
!git clone https://github.com/pmocz/sph-python.git

Cloning into 'sph-python'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 10), reused 14 (delta 5), pack-reused 0
Receiving objects: 100% (24/24), 163.05 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (10/10), done.


#**Loading sph.py File**

Evitando aumentar la extensión del notebook, importamos las siguientes funciones declaradas dentro del archivo `sph.py`.

- `W( x, y, z, h)` = Gaussian Smoothing Kernel
- `gradW( x, y, z, h)` = Gradient of Gaussian Smoothing Kernel
- `getPairwiseSeparations( ri, rj)` = Separation between Particles
- `getDensity( r, pos, m, h)` = Density Function
- `getPressure(rho, k, n)` = Pressure Function
- `getAcc( pos, vel, m, h, k, n, lmbda, nu)` = Acceleration Function

In [2]:
#@markdown Importing SPH Functions
%cd sph-python
import sph as sph

/content/sph-python




---


#**Importing Libraries**

In [3]:
# @title

#Data Management
import numpy as np

#Force Constant Computation
from scipy.special import gamma

#Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Spherical to Cartesian Coordinates
from astropy.coordinates import spherical_to_cartesian



---

#**Solution**

Definimos los siguientes parámetros:

- `dt` = Integration Step

- `epsilon` = Tolerance Condition

- `M_star` = Star Mass

- `R_star` = Star Radius

- `N_particles` = Number of Particles

- `m_particles` =  Particle Mass ($m=M/N)$

- `h` =  Smoothing Length

- `k` =  Equation of State Constant

- `n` =  Polytropic Index

- `nu` =  Viscosity

- `lmbda` =  External Force Constant ~ Mocz. P (2011) [1]

  $$\lambda = \frac{2k\,(1+n)\,\pi^{-\frac{3}{2n}}}{R^2}\left(\frac{M\,\Gamma(\frac{5}{2} +n)}{R^3\,\Gamma(1+n)}\right)^{\frac{1}{n}}$$

  [1] Mocz, P. (2011). Smoothed particle hydrodynamics: theory, implementation, and application to toy stars. Monthly Notices of the Royal Astronomical Society, 10(1), 1-9. https://pmocz.github.io/manuscripts/pmocz_sph.pdf

**Nota:** Sientase en libertad de cambiar cualquiera de los parámetros definidos, principalmente el valor de Masa y Radio de la Estrella.

In [4]:
#@markdown Simulation Parameters

#Star Particles Parameters
M_star = 2
R_star = 0.75
N_particles = 500
m_particles = M_star/N_particles

#SPH Parameters
h, k, n, nu = 0.1, 0.1, 1, 1

#Force Constant  [Mocs. P (2011)]
lmbda = 2*k*(1 + n)*np.pi**(-3/(2*n)) / R_star**2 *\
        (M_star*gamma(5/2 + n)/R_star**3/gamma(1 + n))**(1/n)

#Integration Parameters
dt = 0.05
epsilon = 1e-5

properties = dict(m = m_particles,
                  h = h, k = k, n = n,
                  lmbda = lmbda, nu = nu)

Definimos y ejecutamos la función `hydrostatic_equilibrium(dt, properties)` para la integración del sistema de pseudo-partículas bajo el método Leapfrog.

- `properties` = Dictionary containing SPH Parameters

Establecemos la Condición de *Stop* con base en la Velocidad Radial Media de las Partículas

$$\bar{v_\rho} = \frac{\sum_i^N \frac{\mathbf{\hat{r_i}}\cdot\mathbf{\hat{v_i}}}{r_i}}{N}\quad;\quad |\bar{v_\rho}| < \varepsilon$$



In [5]:
#@markdown `hydrostatic_equilibrium` Integration Function

def hydrostatic_equilibrium(dt, properties):

  #Initial Conditions
  t = 0
  pos   = np.random.randn(N_particles, 3)
  vel   = np.zeros(pos.shape)

  #Hydrostatic Equilibrium Condition
  stop = False

  #Integration
  while ~stop:

    t += dt

    a = sph.getAcc(pos, vel, **properties)

    vel += a*dt/2
    pos += vel*dt

    a = sph.getAcc(pos, vel, **properties)

    vel += a*dt/2

    #Radial Velocity Condition
    vel_radial = np.sum(pos*vel, axis=1)/np.linalg.norm(pos, axis=1)

    stop = abs(np.mean(vel_radial)) < epsilon

  return t, pos, vel

In [6]:
#@markdown Executing `hydrostatic_equilibrium` Function
t, pos, vel = hydrostatic_equilibrium(dt, properties)

print(f'Hydrostatic Equilibrium in...\n\nt: {round(t, 2)} [Ut]')

Hydrostatic Equilibrium in...

t: 17.8 [Ut]


De acuerdo con Mocz. P (2011) [1], cuando se alcanza la condición de Equilibrio Hidrostático es posible encontrar soluciones análiticas para los perfiles de Densidad y Presión como función del Radio.

$$p(r) = k\rho^{1+\frac{1}{n}}\qquad\rho(r)=\left(\frac{\lambda}{2k(1+n)}(R^2 - r^2)\right)^n$$

- [1] Mocz, P. (2011). Smoothed particle hydrodynamics: theory, implementation, and application to toy stars. Monthly Notices of the Royal Astronomical Society, 10(1), 1-9. https://pmocz.github.io/manuscripts/pmocz_sph.pdf

In [7]:
#@markdown Computing Density and Pressure Numerical Profiles

#Test Particles Sampling
rs = np.linspace(0, R_star, 100)

rhos = np.zeros((10, 100))
ps = np.zeros((10, 100))

for i, theta in enumerate(np.linspace(-np.pi, np.pi, 10)):

  #Getting Cartesian Positions
  pos_test = np.array(spherical_to_cartesian(rs, theta/2, theta)).T

  #Numerical Values
  rhos[i] = sph.getDensity(pos_test, pos, m_particles, h).T[0]
  ps [i]  = sph.getPressure(rhos[i], k, n)

#Averaging Profiles
rhos = np.mean(rhos, axis=0)
ps = np.mean(ps, axis=0)

In [11]:
#@markdown Computing Density and Pressure Theoretical Profiles

rhos_theory = (lmbda*(R_star**2 - rs**2)/(2*k*(1 + n)))**n

ps_theory = sph.getPressure(rhos_theory, k, n)

A continuación las siguientes conclusiones:

- Verificamos la solución análitica a los perfiles de Densidad y Presión en la Estrella formada dentro del estado de Equilibrio Hidrostático.

- Es posible definir inicialmente el Radio de la Estrella. La simulación evoluciona el sistema apropiadamente hasta alcanzar dicho tamaño.

In [84]:
#@markdown Plotting Profiles and Hydrostatic Equilibrium State

fig1 = go.Figure(data = [go.Scatter(x = rs, y = rhos,
                                    name = 'ρ (SPH)'),
                         go.Scatter(x = rs, y = rhos_theory,
                                    name = 'ρ (Theory)'),
                         go.Scatter(x = rs, y = ps,
                                    name = 'P (SPH)'),
                         go.Scatter(x = rs, y = ps_theory,
                                    name = 'P (Theory)')],

                 layout = go.Layout(xaxis_title = 'r [UL]',
                                    yaxis_title = 'ρ [Uρ] ,  P [UP]'))

fig = make_subplots(rows = 1, cols = 2,
                    specs=[[{'type': 'scene'}, {'type': 'xy'}]],
                    figure = fig1, column_widths=[0.4, 0.6])

fig.add_trace(go.Scatter3d(x = pos[:,0],
                           y = pos[:,1],
                           z = pos[:,2],
                           mode = 'markers',
                           marker_color = np.linalg.norm(pos, axis = 1),
                           marker_colorscale = 'Oranges',
                           marker_size = 3,
                           showlegend = False),
              row = 1, col = 1)

fig.update_layout(width = 1400, height = 600,
                  title_text = 'Pressure and Density Profile<br>' +\
                               f'SPH Simulation (t = {round(t, 2)} [Ut])',
                  title_x = 0.5, title_y = 0.95,
                  legend = dict(orientation = 'h',
                                x = 0.67, y = 1.07))

fig.show()